In [1]:
import os
try:
    os.chdir('Users/giosue.cotugno/mlops_titanic/notebooks/')
except:
    print('already moved')

In [2]:
from azureml.core import Environment, Workspace
from azureml.core.conda_dependencies import CondaDependencies
import sklearn
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core import Model

In [3]:
import pandas as pd

preproc_train=pd.read_csv('../data/train/preproc.csv')
preproc_train.drop(columns='Unnamed: 0',inplace=True)
preproc_train.columns
X = preproc_train.drop("Survived",axis=1)
y = preproc_train["Survived"]

In [4]:
ws= Workspace.from_config()

In [5]:
from azureml.core.model import Model as AMLModel
def get_model(
    model_name: str,
    model_version: int = None,  # If none, return latest model
    tag_name: str = None,
    tag_value: str = None,
    aml_workspace: Workspace = None
) -> AMLModel:
    """
    Retrieves and returns a model from the workspace by its name
    and (optional) tag.

    Parameters:
    aml_workspace (Workspace): aml.core Workspace that the model lives.
    model_name (str): name of the model we are looking for
    (optional) model_version (str): model version. Latest if not provided.
    (optional) tag (str): the tag value & name the model was registered under.

    Return:
    A single aml model from the workspace that matches the name and tag, or
    None.
    """
    if aml_workspace is None:
        print("No workspace defined - using current experiment workspace.")
        raise ValueError(
                "No workspace provided"
            )

    tags = None
    if tag_name is not None or tag_value is not None:
        # Both a name and value must be specified to use tags.
        if tag_name is None or tag_value is None:
            raise ValueError(
                "model_tag_name and model_tag_value should both be supplied"
                + "or excluded"  # NOQA: E501
            )
        tags = [[tag_name, tag_value]]

    model = None
    if model_version is not None:
        # TODO(tcare): Finding a specific version currently expects exceptions
        # to propagate in the case we can't find the model. This call may
        # result in a WebserviceException that may or may not be due to the
        # model not existing.
        model = AMLModel(
            aml_workspace,
            name=model_name,
            version=model_version,
            tags=tags)
    else:
        models = AMLModel.list(
            aml_workspace, name=model_name, tags=tags, latest=True)
        if len(models) == 1:
            model = models[0]
        elif len(models) > 1:
            raise Exception("Expected only one model")

    return model

In [6]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

ml_client= MLClient.from_config(DefaultAzureCredential())

Found the config file in: /config.json


In [7]:
model=get_model("titanic-xgb.pkl",aml_workspace=ws)

In [9]:
environment = Environment('my-sklearn-environment')
environment.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    'pip==20.2.4'],
    pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'azureml_core',
    'xgboost',
    'scikit-learn=={}'.format(sklearn.__version__)
])

In [83]:
model

Model(workspace=Workspace.create(name='mlops-aml-ws', subscription_id='f90533aa-280d-40b9-9949-a7ba0ee9511f', resource_group='mlops-rg'), name=titanic-xgb.pkl, id=titanic-xgb.pkl:5, version=5, tags={'area': 'Titanic_classification', 'run_id': '1f9fdd03-05d1-4508-92f9-237fea39c39d', 'experiment_name': 'evaluation', 'azureml.nodeid': '3d62ba08', 'azureml.pipeline': '1f9fdd03-05d1-4508-92f9-237fea39c39d', 'mlflow.source.type': 'JOB', 'mlflow.source.name': 'train.py', 'dataset_id': '84aecbe0-f2bb-4815-b21c-21bb22ffec6d', 'mlflow.parentRunId': '1f9fdd03-05d1-4508-92f9-237fea39c39d', 'mlflow.rootRunId': '1f9fdd03-05d1-4508-92f9-237fea39c39d', 'mlflow.runName': 'magenta_beach_wm03kcg4', 'mlflow.user': 'Giosuè Cotugno', 'mse': '0.02242152466367713'}, properties={})

In [84]:
#import os
#os.chdir('Users/giosue.cotugno/mlops_titanic/notebooks/')

# Model deploying

In [ ]:



service_name = 'titanic1endpoint'

inference_config = InferenceConfig(entry_script='../src/scoring/score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

In [ ]:
service

AciWebservice(workspace=Workspace.create(name='mlops-aml-ws', subscription_id='f90533aa-280d-40b9-9949-a7ba0ee9511f', resource_group='mlops-rg'), name=titanic1endpoint, image_id=None, image_digest=None, compute_type=ACI, state=Failed, scoring_uri=None, tags=None, properties={'azureml.git.repository_uri': 'git@github.com:cotus997/mlops-titanic.git', 'mlflow.source.git.repoURL': 'git@github.com:cotus997/mlops-titanic.git', 'azureml.git.branch': 'main', 'mlflow.source.git.branch': 'main', 'azureml.git.commit': 'c2e076d2a23cea4735b3947b6066024bdd239d57', 'mlflow.source.git.commit': 'c2e076d2a23cea4735b3947b6066024bdd239d57', 'azureml.git.dirty': 'True'}, created_by={'userObjectId': '0ede9b9c-2b54-4172-a26d-bbeff841eda9', 'userPuId': '10037FFEA30E4C08', 'userIdp': None, 'userAltSecId': None, 'userIss': 'https://sts.windows.net/e99647dc-1b08-454a-bf8c-699181b389ab/', 'userTenantId': 'e99647dc-1b08-454a-bf8c-699181b389ab', 'userName': 'Giosuè Cotugno', 'upn': 'giosue.cotugno@studio.unibo.it'}

In [ ]:
import json
import numpy

input_payload = json.dumps({
    'data': X[0:2].tolist(),
    'method': 'predict_proba'  # If you have a classification model, you can get probabilities by changing this to 'predict_proba'.
})

output = service.run(input_payload)

print(output)

In [ ]:
service.delete()

# Multi model endpoint

In [57]:
model_1=get_model("titanic-xgb.pkl",aml_workspace=ws,model_version=7)
model_2=get_model("my-sklearn-model",aml_workspace=ws,model_version=6)

In [58]:
from azureml.core.webservice import AciWebservice

aci_service_name = "aciservice-multimodel"
inference_config = InferenceConfig(entry_script="../src/scoring/score_multi.py", environment=environment)
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(ws, aci_service_name, [model_1, model_2], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(True)



/tmp/ipykernel_9815/2624093246.py:7: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoint-sdk-v2 /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(ws, aci_service_name, [model_1, model_2], inference_config, deployment_config, overwrite=True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-12-19 17:05:04+00:00 Creating Container Registry if not exists.
2022-12-19 17:05:04+00:00 Registering the environment.
2022-12-19 17:05:05+00:00 Use the existing image.
2022-12-19 17:05:05+00:00 Generating deployment configuration.
2022-12-19 17:05:06+00:00 Submitting deployment to compute.
2022-12-19 17:05:09+00:00 Checking the status of deployment aciservice-multimodel..
2022-12-19 17:07:53+00:00 Checking the status of inference endpoint aciservice-multimodel.
Succeeded
ACI service creation operation finished, operation "Succeeded"


# Test web service

In [76]:
import json
import numpy as np

test_sample = json.dumps({'data': [[0.0,0.0,3.0,25.14,0.0,0.0,8.4583],[0.0,0.0,3.0,25.14,0.0,0.0,8.4583]]})

prediction = service.run(test_sample)

print(prediction)

ConnectionError: HTTPConnectionPool(host='b4dab246-dddf-4d76-a066-265a3de89d38.westeurope.azurecontainer.io', port=80): Max retries exceeded with url: /score (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f4e58ff9190>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [66]:
service.delete()

No service with name aciservice-multimodel found to delete.


# Deploy to AKS

In [8]:
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice

## New environment for monitoring

In [9]:
environment_monitoring = Environment('inference-env-monitoring')
environment_monitoring.docker.base_image = 'mcr.microsoft.com/azureml/minimal-ubuntu18.04-py37-cpu-inference:latest'
environment_monitoring.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    'pip==20.2.4'],
    pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'azureml-monitoring',
    'xgboost',
    'scikit-learn=={}'.format(sklearn.__version__)
])

### Inference config

### Provision the AKS Cluster

In [10]:
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your AKS cluster
aks_name = 'akscluster1' 

# Verify that cluster does not exist already
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # config:
    '''
    "parameters": {
    "location": {
      "value": "westeurope"
    },
    "workspaceName": {
      "value": "mlops-AML-WS"
    },
    "computeInstanceName": {
      "value": "akscompute1"
    },
    "agentCount": {
      "value": 3
    },
    "agentVMSize": {
      "value": "Standard_A2_v2"
    },
    "clusterPurpose": {
      "value": "DevTest"
    }
  }
    '''
    # Use the default configuration (can also provide parameters to customize)
    prov_config = AksCompute.provisioning_configuration(location="westeurope",agent_count=2,vm_size="Standard_A2_v2",cluster_purpose="DevTest")
    
    # Create the cluster
    aks_target = ComputeTarget.create(workspace = ws, 
                                    name = aks_name, 
                                    provisioning_configuration = prov_config)

if aks_target.get_status() != "Succeeded":
    aks_target.wait_for_completion(show_output=True)

InProgress....................................................................
SucceededProvisioning operation finished, operation "Succeeded"


### Deploy model on AKS

In [11]:
model=get_model('titanic-xgb.pkl',aml_workspace=ws,)

In [12]:
inference_config = InferenceConfig(entry_script='../src/scoring/score_monitoring.py', environment=environment_monitoring)

In [13]:
aks_config = AksWebservice.deploy_configuration(collect_model_data=True, enable_app_insights=True)
aks_service_name ='aks-service-1'

aks_service = Model.deploy(workspace=ws,
                           name=aks_service_name,
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target,
                           overwrite=True)

aks_service.wait_for_deployment(show_output = True)
print(aks_service.state)

/tmp/ipykernel_10001/206965663.py:4: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoint-sdk-v2 /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration. 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  aks_service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-12-21 12:13:15+00:00 Creating Container Registry if not exists.
2022-12-21 12:13:15+00:00 Registering the environment.
2022-12-21 12:13:16+00:00 Use the existing image.
2022-12-21 12:13:18+00:00 Creating resources in AKS.
2022-12-21 12:13:19+00:00 Submitting deployment to compute.
2022-12-21 12:13:19+00:00 Checking the status of deployment aks-service-1..
2022-12-21 12:16:59+00:00 Checking the status of inference endpoint aks-service-1.
Succeeded
AKS service creation operation finished, operation "Succeeded"
Healthy


In [20]:
import json

test_sample = json.dumps({'data':[ [1.0,0.0,1.0,54.0,0.0,0.0,51.8625]]})
#,[0.0,0.0,3.0,25.14,0.0,0.0,8.4583],[1.0,0.0,1.0,54.0,0.0,0.0,51.8625] [0.0,0.0,3.0,25.14,0.0,0.0,8.4583]

prediction = aks_service.run(test_sample)

print(prediction)

[0]


In [27]:
aks_service.delete()

In [84]:
aks_service.delete()
aks_target.delete()